## Messari standardised subgraphs

https://github.com/messari/subgraphs

### Platypus Finance

06/09/2022

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [2]:
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/platypus-finance-avalanche'

In [3]:
#Check if subgraph is up to date
query = """{
  _meta{
    block{
      timestamp
  		hash
      number
    }
    hasIndexingErrors
    deployment
  }
}"""
request = requests.post(endpoint, json={'query':query})
data = request.json()
#Check if it is ok to run i.e. within 5 days of today
print(dt.datetime.fromtimestamp(int(data['data']['_meta']['block']['timestamp'], base = 0)))
int(data['data']['_meta']['block']['timestamp'], base = 0) > (dt.datetime.today().timestamp() - 3600*24*5)

2022-09-06 17:50:13


True

6-Month Average Daily Volume

In [4]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [5]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [6]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [7]:
df['dailyVolumeUSD'].astype(float).mean()

45188088.54618844

6-Month Average TVL

In [8]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/platypus-finance.csv")

In [9]:
lma

,Unnamed: 0,Date,Timestamp,Platypus Finance,Platypus Finance.1,Platypus Finance.2,Platypus Finance.3,Platypus Finance.4,Platypus Finance.5,Platypus Finance.6,...,Platypus Finance.66,Platypus Finance.67,Platypus Finance.68,Platypus Finance.69,Platypus Finance.70,Platypus Finance.71,Platypus Finance.72,Platypus Finance.73,Platypus Finance.74,Platypus Finance.75
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,...,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,avax-staking,avax,Total,staking,avax-staking,avax,avax,...,Total,Total,Total,Total,Total,Total,Total,Total,Total,staking
2,NaN,NaN,NaN,TVL,TVL,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),...,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTP,usd-coin,dai,...,benqi-liquid-staked-avax,yusd-stablecoin,moremoney-usd,defrost-finance-h2o,teddy-dollar,wrapped-bitcoin,bitcoin,token-dforce-usd,mimatic,PTP
4,NaN,27/11/2021,1.637971e+09,NaN,3.9837163796855206,3.9837163796855206,NaN,NaN,0.9958901396979307,0.8961246084872331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,NaN,02/09/2022,1.662077e+09,3537508.9696843163,159081937.23238838,159081937.23238838,3537508.9696843163,3537508.9696843163,46706952.7329722,4940111.5386691075,...,1202496.9773897943,11557327.44296781,568993.6966750415,373249.859398378,NaN,393.68729257,926.3322795,161036.7123429545,20.1770106510217,25228276.777095392
284,NaN,03/09/2022,1.662163e+09,3378082.8254284966,158421577.12304708,158421577.12304708,3378082.8254284966,3378082.8254284966,49349623.387679,3996374.957709841,...,1209990.2251700887,9739675.576874007,563838.6174394283,380092.17245942575,NaN,461.98897369,906.05049259,161036.7123429545,20.1770106510217,25251596.50334883
285,NaN,04/09/2022,1.662250e+09,3446865.9911617776,158053323.56135488,158053323.56135488,3446865.9911617776,3446865.9911617776,48810405.82913001,4037824.874488825,...,1209864.1352262113,9809004.965114493,564646.6948200135,380092.17245942575,NaN,420.20705461,938.11108986,161036.7123429545,20.1770106510217,25285664.965938054
286,NaN,05/09/2022,1.662336e+09,3426150.124299774,156877044.74587163,156877044.74587163,3426150.124299774,3426150.124299774,48531570.842327,3972997.197821643,...,1213707.5889710765,9603030.791800536,565106.6948200135,380092.17245942575,NaN,421.85143281,879.34862661,161036.7123429545,20.1770106510217,25331791.441836096


In [10]:
lma.iloc[:,5]

0                   Dexes
1                   Total
2                     TVL
3                     NaN
4      3.9837163796855206
              ...        
283    159081937.23238838
284    158421577.12304708
285    158053323.56135488
286    156877044.74587163
287    157749942.43226027
Name: Platypus Finance.2, Length: 288, dtype: object

In [11]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 5].astype(float).mean()

471277225.77828777

Largest Pool Concentration

In [12]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_128449/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [13]:
df

,id,name,totalValueLockedUSD,cumulativeVolumeUSD,createdTimestamp,createdBlockNumber
0,0x0d26d103c91f63052fbca88aaf01d5304ae40015,USDT.e on Main Pool,6865494.511021967872,3265876987.869970722727,1637845026,7410081
1,0x909b0ce4fac1a0dca78f8ca7430bbafeeca12871,USDC.e on Main Pool,22092198.164583427296,4841329720.96298008044,1637845465,7410300
2,0xc1daa16e6979c2d1229cb1fd0823491ea44555be,DAI.e on Main Pool,7925208.301696702933438762073033,1773767392.525400157667775230621445,1637846718,7410924
3,0x6220baad9d08dee465befae4f82ee251cf7c8b82,MIM on Withdraw Pool MIM,186001880.294330610651378929408598,509360463.475508421915974722361434,1637846786,7410957
4,0xaef735b1e7ecfaf8209ea46610585817dc0a2e16,USDC on Main Pool,33734848.499308742952,4851897834.124768286564,1643907045,10438732
5,0x776628a5c37335608dd2a9538807b9bba3869e14,USDt on Main Pool,6766941.174658576664,2997674411.559236973313,1645520525,11243940
6,0x6fd4b4c38ed80727ecd0d58505565f9e422c965f,FRAX on Alt Pool Frax,924787.36488422055678703626454,67164769.250500393566921012021118,1649163070,13024690
7,0x035d7d7f209b5d18e2ab5c2072e85b32e1d43760,USDC on Alt Pool Frax,265673.073220558812,67218657.879271544286,1649163078,13024694
8,0xc7388d98fa86b6639d71a0a6d410d5cdfc63a1d0,UST on Withdraw Pool UST,2441476.583357000916,450648420.743584442959,1649163100,13024705
9,0xfc95481f79ec965a535ed8cef4630e1dd308d319,USDC on Withdraw Pool UST,0,450796938.157855765395,1649163110,13024706


In [14]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.5389071166232385

In [15]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.9373856438133431

Liquidity Pools with > $1mn TVL

In [16]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

14

Monthly Active Users

In [17]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [18]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [19]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [23]:
df['dailyActiveUsers'].astype(float).mean()

346.08196721311475

In [20]:
df['date'] = df['timestamp'].astype(int).apply(lambda x: dt.datetime.fromtimestamp(x))

In [21]:
monthactiveusers = []
for i in range(len(df['date'].dt.month.unique())):
    monthactiveusers.append(df['dailyActiveUsers'].loc[df['date'].dt.month == df['date'].dt.month.unique()[i]].astype(float).sum())

In [22]:
np.mean(monthactiveusers) #can chart this easy

9047.57142857143

Implied slippage